# League of Legends: matches data parser from riot official api
#### This code is able to generate 100 matchIDs and write the data to a csv. 

###### created by:                   Aniruddha Dutta
###### Created on:                   13th Nov 2019
###### last modified on:             13th Nov 2019

In [1]:
import requests
import json
import sys
import time
import pandas as pd
from pandas.io.json import json_normalize

Setting the URL and parameters

In [2]:
URL = 'https://na1.api.riotgames.com/lol/match/v4/matches/3187158923'
gameId = ['3187158923']
params = {'api_key': 'RGAPI-8b589730-3f42-4017-8ece-8a4eba590641'}

### Funtions:

Declaring the following user-defined functions:
1. get_matches_data(): to crawl match data(json)for the set of gameIds from the riot match api
2. get_player_ids(): to crawl list of participants and their accountIds/PlayerIds
3. get_game_ids(): get the 10 most recent gameIds for each individual playerId

In [34]:
def get_matches_data(gameId, params):
    
    c = 1
    
    for game in gameId:
        #set url for fetching match data from the riot match api
        url = 'https://na1.api.riotgames.com/lol/match/v4/matches/'+ str(game)
        print('GameId:' + str(game))
        #fetch match json for the given url
        r = requests.get(url = url, params = params)
        print('request_status: '+ str(r))
        data = json.loads(json.dumps(r.json()))
        
        if bool(data['teams'][0]['bans'])== True and r.status_code == 200:
            df = json_normalize(data) #normalize json into dataframe to flatten data
            
            #flatten matches json data
            #seperate the metadata columns
            df.drop(['teams', 'participants','participantIdentities'], axis = 1, inplace = True) 
            metadata = df
            #flatten the participants and participantIdentities into different datasets and merge to get the participant data
            participants_df = json_normalize(data, 
                                             record_path = 'participants',
                                             meta = 'gameId')
            
            participantId_df = json_normalize(data,
                                              record_path = 'participantIdentities',
                                              meta = 'gameId')
            
            participant_data = participants_df.merge(participantId_df,
                                                     left_on = ['gameId','participantId'],
                                                     right_on = ['gameId','participantId'])
            #flatten Teams data to get team level statistics and champion bans per team
            team_data = json_normalize(data,
                                       record_path = 'teams',
                                       meta = 'gameId')
            team_data.drop('bans', axis = 1, inplace = True)
            
            team_bans = json_normalize(data['teams'],
                                       record_path = 'bans',
                                       meta = 'teamId')
            #renaming column to state that these championIDs refer to thise champions that were banned by each team
            team_bans.rename(columns = {'championId' : 'championId_bans'}, inplace = True) 
            
            teams_data = team_data.merge(team_bans, 
                                         left_on = 'teamId',
                                         right_on = 'teamId')
            
            #merging the prepared data for participants, teams and metadata to create the final match data
            match_data = (participant_data.merge(teams_data,
                                                 left_on = ['gameId', 'teamId', 'participantId'],
                                                 right_on = ['gameId', 'teamId','pickTurn'])).merge(df, 
                                                                                                    left_on = 'gameId',
                                                                                                    right_on = 'gameId')
            match_data.drop(['stats.participantId',
                             'stats.longestTimeSpentLiving',
                             'stats.doubleKills',
                             'stats.tripleKills',
                             'stats.quadraKills',
                             'stats.pentaKills',
                             'stats.unrealKills',
                             'stats.perk0Var1',
                             'stats.perk0Var2',
                             'stats.perk0Var3',
                             'stats.perk1Var1',
                             'stats.perk1Var2',
                             'stats.perk1Var3',
                             'stats.perk2Var1',
                             'stats.perk2Var2',
                             'stats.perk2Var3',
                             'stats.perk3Var1',
                             'stats.perk3Var2',
                             'stats.perk3Var3',
                             'stats.perk4Var1',
                             'stats.perk4Var2',
                             'stats.perk4Var3',
                             'stats.perk5Var1',
                             'stats.perk5Var2',
                             'stats.perk5Var3',
                             'timeline.participantId',
                             'player.platformId',
                             'player.currentPlatformId',
                             'player.currentAccountId',
                             'player.profileIcon'], axis = 1, inplace = True)
            print('match data pull success')
            if c==1:
                match_final_data = match_data
            else:
                match_final_data = match_final_data.append(match_data)
            c = c+1
            time.sleep(10)
        elif bool(data['teams'][0]['bans'])== False or r.status_code == 503:
            print('The matchId ' + str(game) + ' does not have bans data')
            time.sleep(10)
        
    return match_final_data

        
    
    

In [24]:
def get_player_id(gameIds, params):
    player_ids = []
    for game in gameIds:
        URL = 'https://na1.api.riotgames.com/lol/match/v4/matches/'+ str(game)
        r = requests.get(url = URL, params = params)
        print(game)
        print('request status: '+ str(r))
        for player_id in r.json()["participantIdentities"]:
            player_ids.append(player_id["player"]["accountId"])
        print('task complete')
        time.sleep(10)
            
    return player_ids

In [29]:
def get_game_id(player_ids, params):
    gameIds = []
    for player in player_ids:
        URL = 'https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/'+ player
        print(URL)
        r = requests.get(url = URL, params = params)
        if r.status_code == 200:
            
            print('request status: ' + str(r))
            #print(json.dumps(r.json()))
            #matches = [x for x in r.json()["matches"]]
            for match in r.json()["matches"]:
                gameIds.append(match["gameId"])
            print('total gameIds: '+ str(len(gameIds)))
            print('task complete')
            time.sleep(10)
        elif r.status_code == 400:
            print('request status: ' + str(r))
    return gameIds

### Main Code:

Currently this section is able to craw data for 100 matchIDs from a single random gameId and store it in a csv format
next steps: write logic to craw more matchIDs from the available information and store them in batchesin csv format.

#### 1. getting data from a single matchID

In [17]:
matches_data = get_matches_data(gameId, params)
matches_data

GameId:3187158923
request_status: <Response [200]>
match data pull success


,participantId,teamId,championId,spell1Id,spell2Id,stats.win,stats.item0,stats.item1,stats.item2,stats.item3,...,pickTurn,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType
0,1,100,91,14,4,True,3117,3142,3077,2031,...,1,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME
1,2,100,81,7,4,True,3153,0,2422,3042,...,2,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME
2,3,100,267,14,4,True,3098,3117,2055,3504,...,3,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME
3,4,100,13,4,12,True,3003,3800,2031,3020,...,4,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME
4,5,100,86,11,4,True,1001,3077,1413,3078,...,5,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME
5,6,200,25,4,14,False,3092,2424,3191,1052,...,6,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME
6,7,200,102,11,4,False,1416,2031,3009,3742,...,7,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME
7,8,200,84,14,4,False,3146,0,0,1001,...,8,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME
8,9,200,122,6,4,False,3078,3077,2033,3111,...,9,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME
9,10,200,51,4,7,False,1055,3031,3006,1042,...,10,NA1,1572164899302,1245,420,11,13,9.21.294.813,CLASSIC,MATCHED_GAME


In [18]:
player_ids = get_player_id(gameId, params)

3187158923
request status: <Response [200]>
task complete


In [19]:
gameIds = get_game_id(player_ids, params)
gameIds

https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/hyvDc1NmXRgPKkA7grs-8i6rTtuNMMIkNBETOGBhnZG4ItM
request status: <Response [200]>
total gameIds: 100
task complete


[3206930808,
 3206868116,
 3206797740,
 3206729537,
 3206722332,
 3206676687,
 3206670762,
 3206635122,
 3206547723,
 3205107586,
 3205105520,
 3205093092,
 3205049091,
 3204995776,
 3204713783,
 3204605411,
 3204591420,
 3204518834,
 3203868403,
 3203863271,
 3203842570,
 3203715463,
 3203723156,
 3203658006,
 3203632409,
 3203606080,
 3203057814,
 3203043929,
 3203026131,
 3202972840,
 3202846074,
 3201763659,
 3201628320,
 3201640405,
 3201575878,
 3201245234,
 3200239119,
 3200158158,
 3200134746,
 3200117089,
 3200121782,
 3200009788,
 3199703096,
 3199652744,
 3199637440,
 3199613197,
 3199558548,
 3199544513,
 3199561437,
 3199522585,
 3199425551,
 3199226212,
 3199212558,
 3199130794,
 3199086917,
 3199082731,
 3199047505,
 3199025127,
 3199001479,
 3199040314,
 3198727520,
 3198722230,
 3197640501,
 3197588299,
 3197544904,
 3197530625,
 3197504318,
 3197426566,
 3197382804,
 3197334244,
 3197258933,
 3197187503,
 3197173271,
 3197075858,
 3196764425,
 3196760479,
 3196626157,

In [20]:
matches_final_data = get_matches_data(gameIds, params)

GameId:3206930808
request_status: <Response [200]>
The matchId 3206930808 does not have bans data
GameId:3206868116
request_status: <Response [200]>
The matchId 3206868116 does not have bans data
GameId:3206797740
request_status: <Response [200]>
The matchId 3206797740 does not have bans data
GameId:3206729537
request_status: <Response [200]>
The matchId 3206729537 does not have bans data
GameId:3206722332
request_status: <Response [200]>
The matchId 3206722332 does not have bans data
GameId:3206676687
request_status: <Response [200]>
The matchId 3206676687 does not have bans data
GameId:3206670762
request_status: <Response [200]>
The matchId 3206670762 does not have bans data
GameId:3206635122
request_status: <Response [200]>
The matchId 3206635122 does not have bans data
GameId:3206547723
request_status: <Response [200]>
match data pull success
GameId:3205107586
request_status: <Response [200]>
match data pull success
GameId:3205105520
request_status: <Response [200]>
match data pull

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


GameId:3204713783
request_status: <Response [200]>
match data pull success
GameId:3204605411
request_status: <Response [200]>
match data pull success
GameId:3204591420
request_status: <Response [200]>
match data pull success
GameId:3204518834
request_status: <Response [200]>
match data pull success
GameId:3203868403
request_status: <Response [200]>
match data pull success
GameId:3203863271
request_status: <Response [200]>
match data pull success
GameId:3203842570
request_status: <Response [200]>
match data pull success
GameId:3203715463
request_status: <Response [200]>
match data pull success
GameId:3203723156
request_status: <Response [200]>
match data pull success
GameId:3203658006
request_status: <Response [200]>
match data pull success
GameId:3203632409
request_status: <Response [200]>
match data pull success
GameId:3203606080
request_status: <Response [200]>
match data pull success
GameId:3203057814
request_status: <Response [200]>
match data pull success
GameId:3203043929
request

In [21]:
matches_final_data.head()

,baronKills,championId,championId_bans,dominionVictoryScore,dragonKills,firstBaron,firstBlood,firstDragon,firstInhibitor,firstRiftHerald,...,timeline.xpDiffPerMinDeltas.10-20,timeline.xpDiffPerMinDeltas.20-30,timeline.xpDiffPerMinDeltas.30-end,timeline.xpPerMinDeltas.0-10,timeline.xpPerMinDeltas.10-20,timeline.xpPerMinDeltas.20-30,timeline.xpPerMinDeltas.30-end,towerKills,vilemawKills,win
0,0,25,114,0,2,False,True,True,False,False,...,-22.15,NaN,NaN,255.3,232.4,NaN,NaN,2,0,Fail
1,0,67,30,0,2,False,True,True,False,False,...,-22.15,NaN,NaN,283.2,438.9,NaN,NaN,2,0,Fail
2,0,90,10,0,2,False,True,True,False,False,...,-25.50,NaN,NaN,440.8,408.7,NaN,NaN,2,0,Fail
3,0,240,8,0,2,False,True,True,False,False,...,-206.90,NaN,NaN,428.8,293.1,NaN,NaN,2,0,Fail
4,0,32,235,0,2,False,True,True,False,False,...,55.90,NaN,NaN,342.7,499.0,NaN,NaN,2,0,Fail


#### 2 Getting data from the new matchIds generated in the above steps

In [26]:
playerId_new = get_player_id(gameIds, params)

3206930808
request status: <Response [200]>
task complete
3206868116
request status: <Response [200]>
task complete
3206797740
request status: <Response [200]>
task complete
3206729537
request status: <Response [200]>
task complete
3206722332
request status: <Response [200]>
task complete
3206676687
request status: <Response [200]>
task complete
3206670762
request status: <Response [200]>
task complete
3206635122
request status: <Response [200]>
task complete
3206547723
request status: <Response [200]>
task complete
3205107586
request status: <Response [200]>
task complete
3205105520
request status: <Response [200]>
task complete
3205093092
request status: <Response [200]>
task complete
3205049091
request status: <Response [200]>
task complete
3204995776
request status: <Response [200]>
task complete
3204713783
request status: <Response [200]>
task complete
3204605411
request status: <Response [200]>
task complete
3204591420
request status: <Response [200]>
task complete
3204518834
req

In [27]:
len(playerId_new)

1000

In [30]:
gameIds_new = get_game_id(playerId_new, params)
len(gameIds_new)

https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/mvRpC8j31XOYozlzTzELc2UbjKEJCQYxxXxKOj-Xbu0
request status: <Response [200]>
total gameIds: 100
task complete
https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/EbCsdEq_o6gcWhcyUzx4mokufNG6FGXgjY2JnNuCsR4wWg
request status: <Response [200]>
total gameIds: 200
task complete
https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/8Kc_RlJmL9B_7IV8CM344j5p2f3sv8ByuO_jL8MwOb3wKA
request status: <Response [200]>
total gameIds: 300
task complete
https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/sVuoE0BRNW1GUBM-YuONmJCfCpzz1ideNRwsWrwWKPyQXg
request status: <Response [200]>
total gameIds: 400
task complete
https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/hyvDc1NmXRgPKkA7grs-8i6rTtuNMMIkNBETOGBhnZG4ItM
request status: <Response [200]>
total gameIds: 500
task complete
https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/RCOCYf4OTh4KnWgJ38bgmTzFg0Lc8Sgr3hs7fQ11ETFV2TE

96712

In [35]:
matches_data = get_matches_data(gameIds_new, params)
matches_data.head()

GameId:3206989874
request_status: <Response [200]>
The matchId 3206989874 does not have bans data
GameId:3206918072
request_status: <Response [200]>
The matchId 3206918072 does not have bans data
GameId:3206905420
request_status: <Response [200]>
The matchId 3206905420 does not have bans data
GameId:3206930808
request_status: <Response [200]>
The matchId 3206930808 does not have bans data
GameId:3206771323
request_status: <Response [200]>
The matchId 3206771323 does not have bans data
GameId:3206745451
request_status: <Response [200]>
The matchId 3206745451 does not have bans data
GameId:3206655725
request_status: <Response [200]>
The matchId 3206655725 does not have bans data
GameId:3206650485
request_status: <Response [200]>
The matchId 3206650485 does not have bans data
GameId:3206624703
request_status: <Response [200]>
match data pull success
GameId:3206586121
request_status: <Response [200]>
The matchId 3206586121 does not have bans data
GameId:3206532852
request_status: <Response

KeyError: 'teams'